# __Delsys FileReaderAPI - C# .NET Overview__
This document outlines how to directly read SHPF files within a .NET Interactive environment using Delsys' FileReaderAPI

## __Requirements__
1) .NET 6 SDK https://dotnet.microsoft.com/download/dotnet/6.0
2) .NET Interactive Notebooks https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode


## Load File Reader Dll from Trigno Discover's Program files
Loads the dll file and Plotly.NET needed to perform the reading and plotting/graphing task.

In [ ]:
#r "Delsys.FileManager.dll"
#r "nuget: Plotly.NET, 2.0.0"
#r "nuget: Microsoft.EntityFrameworkCore.Design, 5.0.17"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite, 5.0.17"
#r "nuget: Microsoft.EntityFrameworkCore.Tools, 5.0.17"

## Setup the File Reader

First import File Reader, then initalize the DotDelsysFileReader by passing the shpf/.delsys file location.

In [ ]:
using FileReader;
using FileReader.Shpf.Models;
using Delsys.FileManager.Reader;

string shpffile = "Forearm_Pronation_Supination_1.delsys";
DelsysFileReader reader = new DelsysFileReader(shpffile);
IReadOnlyList<DelsysTrial> files = reader.OpenFile();

## Navigating The File
You can loop through trials, the sensors within the trial, and the channels within the sensor, by using indexing

In [ ]:
using Delsys.FileManager.FileRepository.Reader.TrialFiles;

string gap = "—— ";
foreach (DelsysTrial trial in files) {
    System.Console.WriteLine("Trial: " + trial.Trial.Name);
    
    foreach (TrialComponent component in trial.Trial.Components) {
        System.Console.WriteLine(gap + "Component: " + component.Name);
        
        foreach (TrialChannel channel in component.Channels) {
            System.Console.WriteLine(gap + gap + "Channel: " + channel.Name);
        }
    }
}

## Retrieving Data
After the file is parsed all of the individual sensor component objects are stored in a list. The file provided with this example only has one sensor's data, therefore we will reference index 0 to retrieve the first component in that list. If more sensors were used during a collection, you may loop through the sensor objects by using the total number of sensors in the file. To see all of the metadata and methods associated with each component object see IFileComponent Interface located at FileReader.Interface

In [ ]:
DelsysTrial file = files[0]; 
List<TrialComponent> components = file.Trial.Components;

TrialComponent component1 = components[0];
System.Console.WriteLine("Component Name: " + component1.Name);

## Get data from a single channel 
Each component object contains a list of channel objects. This example will pull the first channel from component1 object. Each channel object has a variety of metadata along with all of the channel data. To see all of the metadata and methods associated with each channel object see IFileChannel Interface located at FileReader.Interface

In [ ]:
TrialChannel channel1 = component1.Channels[0];
System.Console.WriteLine("Channel Name: " + channel1.Name);

## Get all data from a component
You can get all channel data (for a single sensor component) and other metadata with these method calls.

In [ ]:
List<double> xseries = file.Trial.GetChannelTimeSeries(channel1.GuidString);
Dictionary<Guid, (ushort Offset, List<double> Samples)>[] channelData = file.Trial.DataStream.GetData(0, file.Trial.DataStream.TotalNumConcatenatedFrames, new Guid[] {channel1.Guid});
List<double> yseries = new List<double>();

for(int i=0; i < channelData.Count(); i++){
    foreach (double sample in channelData[i][channel1.Guid].Samples){
        yseries.Add(sample);
    }
}

string channelName = channel1.Name;
double channelSampleRate = channel1.SampleRate;
string channelUnits = Enum.GetName(typeof(FileReader.Shpf.Enums.Unit), channel1.Units);

System.Console.WriteLine("Channel: " + channelName);
System.Console.WriteLine("  Sample Rate: " + channelSampleRate.ToString());
System.Console.WriteLine("  Units: " + channelUnits);
System.Console.WriteLine("  Data Length: " + yseries.Count.ToString());


## Plotting single channel of data
Here the EMG data is plotted on its own figure

In [ ]:
using Plotly.NET;
using Plotly.NET.LayoutObjects;

// Calculates the time axis for first channel based on channel sample rate

// Set x-axis
LinearAxis xAxis = new LinearAxis();
xAxis.SetValue("title", "Time (s)");

// Set y-axis
LinearAxis yAxis = new LinearAxis();
yAxis.SetValue("title", Enum.GetName(typeof(FileReader.Shpf.Enums.Unit), channel1.Units));

// Create plot layout
Layout layout = new Layout();
layout.SetValue("xaxis", xAxis);
layout.SetValue("yaxis", yAxis);
layout.SetValue("title", channel1.Name);

// Set plot data
Trace trace = new Trace("scatter");
trace.SetValue("x", xseries);
trace.SetValue("y", yseries);
trace.SetValue("mode", "lines");

// Show plot
GenericChart
    .ofTraceObject(true, trace)
    .WithLayout(layout)
    .Show();

## Plot all data from sensor component
Here all of the data is plotted based on the channel type (EMG, ACC, GYRO)

In [ ]:
List<string> channel_types = new List<string>();
for (int i=0;i<component1.Channels.Count(); i++){
    if (channel_types.Contains(component1.Channels[i].ChannelType.ToString())){
        continue;
    }
    else{
        channel_types.Add(component1.Channels[i].ChannelType.ToString());
    }
}

List<List<GenericChart.GenericChart>> plots = new List<List<GenericChart.GenericChart>>();

//Loop all unique channel types
for (int i=0; i<channel_types.Count(); i++)
{
    List<GenericChart.GenericChart> uniqueTypePlot = new List<GenericChart.GenericChart>();

    LinearAxis xAxis = new LinearAxis();
    xAxis.SetValue("title", "Time (s)");
    
    //Loop all channel data - if channel type matches, add the channel data to that plot
    for (int k=0; k<component1.Channels.Count(); k++)
    {
        if (component1.Channels[k].ChannelType.ToString() == channel_types[i])
        {
            Dictionary<Guid, (ushort Offset, List<double> Samples)>[] channelData = file.Trial.DataStream.GetData(0, file.Trial.DataStream.TotalNumConcatenatedFrames, new Guid[] {component1.Channels[k].Guid});
            List<double> yseries = new List<double>();

            for(int j=0; j < channelData.Count(); j++){
                foreach (double sample in channelData[j][component1.Channels[k].Guid].Samples){
                    yseries.Add(sample);
                }
            }

            Trace trace = new Trace("scatter");
            trace.SetValue("x", xseries);
            trace.SetValue("y", yseries);
            trace.SetValue("mode", "lines");
            
            trace.SetValue("name", component1.Channels[k].Name);

            LinearAxis yAxis = new LinearAxis();
            yAxis.SetValue("title", Enum.GetName(typeof(FileReader.Shpf.Enums.Unit), component1.Channels[k].Units));

            Layout layout = new Layout();
            layout.SetValue("xaxis", xAxis);
            layout.SetValue("yaxis", yAxis);
            layout.SetValue("showlegend", true);
            layout.SetValue("title", channel_types[i]);

            var plotPart = GenericChart.ofTraceObject(true, trace).WithLayout(layout);
            uniqueTypePlot.Add(plotPart);
        }
    }
    plots.Add(uniqueTypePlot);
}

for (int i = 0; i<plots.Count(); i++)
{
    var plot = Plotly.NET.Chart.Combine(plots[i]);
    plot.Show();
}

## Close File


In [ ]:
reader.Close()